Data Modelling and Detection

Data processing for model

In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [9]:
#loading CSV file which is preprocessed as a DataFrame
df = pd.read_csv('.\Datasets\preprocessed_Data.csv')
df.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,target,attack_type
0,0,1,0,181,5450,0,0,0,0,0,...,1.0,0.0,0.0,9,9,0.0,0.11,0.0,normal.,normal
1,0,1,0,239,486,0,0,0,0,0,...,1.0,0.0,0.0,19,19,0.0,0.05,0.0,normal.,normal
2,0,1,0,235,1337,0,0,0,0,0,...,1.0,0.0,0.0,29,29,0.0,0.03,0.0,normal.,normal
3,0,1,0,219,1337,0,0,0,0,0,...,1.0,0.0,0.0,39,39,0.0,0.03,0.0,normal.,normal
4,0,1,0,217,2032,0,0,0,0,0,...,1.0,0.0,0.0,49,49,0.0,0.02,0.0,normal.,normal


In [10]:
df['target'].unique()

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

In [11]:
corr_matrx = df.iloc[:,:-2].corr()

In [12]:
#Filter the correlation matrix to show only values greater than 0.8
high_corr = corr_matrx[corr_matrx > 0.8]

#Remove self-correlation
for i in range(len(high_corr)):
    high_corr.iat[i,i] = None

#Drop rows and columns where there are no values > 0.8
high_corr_filtered = high_corr.dropna(how='all').dropna(axis=1,how='all')

high_corr_filtered

,hot,is_guest_login,count,srv_count,same_srv_rate,dst_host_srv_count,dst_host_same_src_port_rate
hot,NaN,0.843572,NaN,NaN,NaN,NaN,NaN
is_guest_login,0.843572,NaN,NaN,NaN,NaN,NaN,NaN
count,NaN,NaN,NaN,0.943667,NaN,NaN,0.860579
srv_count,NaN,NaN,0.943667,NaN,NaN,NaN,0.944926
same_srv_rate,NaN,NaN,NaN,NaN,NaN,0.898955,NaN
dst_host_srv_count,NaN,NaN,NaN,NaN,0.898955,NaN,NaN
dst_host_same_src_port_rate,NaN,NaN,0.860579,0.944926,NaN,NaN,NaN


In [13]:
df.drop(['is_guest_login','srv_count','dst_host_srv_count'],axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
df.attack_type.value_counts()

attack_type
dos       391458
normal     97278
probe       4107
r2l         1126
u2r           52
Name: count, dtype: int64

In [15]:
df.isnull().sum()

duration                       0
protocol_type                  0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
count                          0
serror_rate                    0
rerror_rate                    0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
dst_host_count                 0
dst_host_diff_srv_rate         0
dst_host_same_src_port_rate    0
dst_host_srv_diff_host_rate    0
target                         0
attack_typ

In [16]:
#Checking for data imbalance
from sklearn.utils import resample

#Seperate majority and minority classes
data_majority = df[df.attack_type == 'dos']
data_minority_5 = df[df['attack_type'] == 'normal']
data_minority_2 = df[df['attack_type'] == 'probe']
data_minority_3 = df[df['attack_type'] == 'r2l']
data_minority_4 = df[df['attack_type'] == 'u2r']

data_minority_5_upsampled = resample(data_minority_5,replace=True,n_samples=len(data_majority),random_state=42)
data_minority_2_upsampled = resample(data_minority_2,replace=True,n_samples=len(data_majority), random_state=42)
data_minority_3_upsampled = resample(data_minority_3,replace=True,n_samples=len(data_majority), random_state=42)
data_minority_4_upsampled = resample(data_minority_4,replace=True,n_samples=len(data_majority), random_state=42)


data_balanced = pd.concat([data_majority,data_minority_5_upsampled,data_minority_2_upsampled,data_minority_3_upsampled,data_minority_4_upsampled])
data_balanced.head()

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,target,attack_type
7601,0,1,1,0,0,0,0,0,0,0,...,0.0,0.83,0.33,0.0,5,0.00,0.20,0.33,neptune.,dos
7602,0,1,1,0,0,0,0,0,0,0,...,0.0,0.94,0.12,0.0,15,0.00,0.07,0.12,neptune.,dos
7793,0,0,0,1032,0,0,0,0,0,0,...,0.0,1.00,0.00,0.0,148,0.02,0.02,0.00,smurf.,dos
7794,0,0,0,1032,0,0,0,0,0,0,...,0.0,1.00,0.00,0.0,158,0.02,0.08,0.00,smurf.,dos
7795,0,0,0,1032,0,0,0,0,0,0,...,0.0,1.00,0.00,0.0,168,0.02,0.14,0.00,smurf.,dos


In [17]:
data_balanced['attack_type'].value_counts()

attack_type
dos       391458
normal    391458
probe     391458
r2l       391458
u2r       391458
Name: count, dtype: int64

In [18]:
df.attack_type.isnull().sum()

0

Test Train split

In [19]:
df['Combined'] = df['target'].astype(str) + "_" + df['attack_type'].astype(str)

# Initialize the Label Encoder
label = LabelEncoder()

# Encode the combined labels
df['Encoded'] = label.fit_transform(df['Combined'])

# Check the encoded labels
print("Encoded labels:", df['Encoded'].unique())

# Prepare features (X) and target variable (y)
X = df.drop(['target', 'attack_type', 'Combined', 'Encoded'], axis=1)  # Drop the original and combined columns
y = df['Encoded']  # Use the encoded column as the target variable

# Scale features
sc = MinMaxScaler()
X = sc.fit_transform(X)

# Split test and train data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print("Training data shape:", X_train.shape, y_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Encoded labels: [11  1  7 12  9 18  3 14 20 15  5  6  2  0  4 17 13 10  8 22 21 19 16]
Training data shape: (330994, 29) (330994,)
Test data shape: (163027, 29) (163027,)


In [20]:
df.attack_type.unique()

array(['normal', 'u2r', 'dos', 'r2l', 'probe'], dtype=object)

In [21]:
#Get all mappings
attack_mapping = {index: name for index,name in enumerate(label.classes_)}

print("Mapping of encoded values to original values: ", attack_mapping)

Mapping of encoded values to original values:  {0: 'back._dos', 1: 'buffer_overflow._u2r', 2: 'ftp_write._r2l', 3: 'guess_passwd._r2l', 4: 'imap._r2l', 5: 'ipsweep._probe', 6: 'land._dos', 7: 'loadmodule._u2r', 8: 'multihop._r2l', 9: 'neptune._dos', 10: 'nmap._probe', 11: 'normal._normal', 12: 'perl._u2r', 13: 'phf._r2l', 14: 'pod._dos', 15: 'portsweep._probe', 16: 'rootkit._u2r', 17: 'satan._probe', 18: 'smurf._dos', 19: 'spy._r2l', 20: 'teardrop._dos', 21: 'warezclient._r2l', 22: 'warezmaster._r2l'}


In [22]:
df

,duration,protocol_type,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,target,attack_type,Combined,Encoded
0,0,1,0,181,5450,0,0,0,0,0,...,0.0,0.00,9,0.0,0.11,0.00,normal.,normal,normal._normal,11
1,0,1,0,239,486,0,0,0,0,0,...,0.0,0.00,19,0.0,0.05,0.00,normal.,normal,normal._normal,11
2,0,1,0,235,1337,0,0,0,0,0,...,0.0,0.00,29,0.0,0.03,0.00,normal.,normal,normal._normal,11
3,0,1,0,219,1337,0,0,0,0,0,...,0.0,0.00,39,0.0,0.03,0.00,normal.,normal,normal._normal,11
4,0,1,0,217,2032,0,0,0,0,0,...,0.0,0.00,49,0.0,0.02,0.00,normal.,normal,normal._normal,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,1,0,310,1881,0,0,0,0,0,...,0.0,0.40,86,0.0,0.01,0.05,normal.,normal,normal._normal,11
494017,0,1,0,282,2286,0,0,0,0,0,...,0.0,0.00,6,0.0,0.17,0.05,normal.,normal,normal._normal,11
494018,0,1,0,203,1200,0,0,0,0,0,...,0.0,0.17,16,0.0,0.06,0.05,normal.,normal,normal._normal,11
494019,0,1,0,291,1200,0,0,0,0,0,...,0.0,0.17,26,0.0,0.04,0.05,normal.,normal,normal._normal,11


In [23]:
def calculate_tp_tn_fp_fn(confusion_matrix):
    """
    Function to calculate True Positives (TP), True Negatives (TN),
    False Positives (FP), and False Negatives (FN) from a given confusion matrix.

    Args:
    confusion_matrix (numpy.ndarray): The confusion matrix (multiclass)

    Returns:
    dict: A dictionary containing TP, TN, FP, and FN values
    """
    # True Negatives (TN): Normal correctly classified as Normal (Class 0 classified as Class 0)
    TN = confusion_matrix[0, 0]

    # False Positives (FP): Normal misclassified as any Attack (Class 0 misclassified as Class 1-4)
    FP = confusion_matrix[0, 1:].sum()

    # True Positives (TP): Attack correctly classified as Attack (Class 1-4 classified as Class 1-4)
    TP = confusion_matrix[1:, 1:].sum()

    # False Negatives (FN): Attack misclassified as Normal (Class 1-4 misclassified as Class 0)
    FN = confusion_matrix[1:, 0].sum()

    return {
        'True Positives (TP)': TP,
        'True Negatives (TN)': TN,
        'False Positives (FP)': FP,
        'False Negatives (FN)': FN
    }

Training Models

Gaussian Naive Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score

In [25]:
clfb = GaussianNB()
start_time_train = time.time()

In [26]:
clfb.fit(X_train, y_train)
end_time_train = time.time()

In [27]:
print(f"Training time: {end_time_train - start_time_train:.3f} seconds")

Training time: 0.585 seconds


In [28]:
start_time_train_pred = time.time()
y_train_pred = clfb.predict(X_train)
end_time_train_pred = time.time()

In [29]:
start_time_test_pred = time.time()
y_test_pred = clfb.predict(X_test)
end_time_test_pred = time.time()

In [30]:
print(f"Prediction time on test data: {end_time_test_pred - start_time_test_pred:.4f} seconds")

Prediction time on test data: 3.0898 seconds


In [31]:
print("Train score is:", clfb.score(X_train, y_train))
print("Test score is:",clfb.score(X_test,y_test))

Train score is: 0.8569460473603752
Test score is: 0.8567660571561766


In [32]:
#Calculate metrics
precision_gaus = precision_score(y_test, y_test_pred, average='weighted')
accuracy_gaus = accuracy_score(y_test, y_test_pred)
recall_gaus = recall_score(y_test,y_test_pred, average='weighted')
f1_gaus = f1_score(y_test,y_test_pred, average='weighted')
cm_gaus = confusion_matrix(y_test,y_test_pred)
r = calculate_tp_tn_fp_fn(cm_gaus)

# Display results
print(f"Precision: {precision_gaus}")
print(f"Accuracy: {accuracy_gaus}")
print(f"Recall: {recall_gaus}")
print(f"F1 Score: {f1_gaus}")
print(f"Confusion Matrix:\n{cm_gaus}")
print(r)


C:\Users\WIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.991411704410934
Accuracy: 0.8567660571561766
Recall: 0.8567660571561766
F1 Score: 0.8849105459621383
Confusion Matrix:
[[  716     0     0     0     4     0     0     0     0     0     0     2
      0     0     0     0     0     0     0     0     0]
 [    0     5     0     0     0     0     0     2     3     0     0     1
      0     0     0     0     0     0     0     0     2]
 [    0     0     1     0     0     1     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0]
 [    0     0     0    15     0     0     0     0     0     0     0     0
      0     0     0     1     0     0     0     0     1]
 [    0     0     0     0     2     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     1]
 [    0     0     0     0     0   412     0     1     0     0     0     2
      0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     6     0     0     0     0     0
      0 

Decision Tree Classifier

In [33]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the model
clfd = DecisionTreeClassifier(criterion="entropy", max_depth=4)

# Train the model
start_time = time.time()
clfd.fit(X_train, y_train)
end_time = time.time()
print("Training time: ", end_time - start_time)

# Predict using the training data
y_train_pred = clfd.predict(X_train)

# Predict using the test data
y_test_pred = clfd.predict(X_test)

#Calculate and display metrics for training and testing
train_accuracy = clfd.score(X_train, y_train)
test_accuracy = clfd.score(X_test, y_test)
print("Train score is:", train_accuracy)
print("Test score is:", test_accuracy)

# Calculate metrics
precision_dec = precision_score(y_test, y_test_pred, average='weighted')
accuracy_dec = accuracy_score(y_test, y_test_pred)
recall_dec = recall_score(y_test, y_test_pred, average='weighted')
f1_dec = f1_score(y_test, y_test_pred, average='weighted')
cm_dec = confusion_matrix(y_test, y_test_pred)
r = calculate_tp_tn_fp_fn(cm_dec)

# Display results
print(f"Precision: {precision_dec}")
print(f"Accuracy: {accuracy_dec}")
print(f"Recall: {recall_dec}")
print(f"F1 Score: {f1_dec}")
#print(f"Confusion Matrix:\n{cm_dec}")
print(r)

Training time:  1.1247806549072266
Train score is: 0.9870209127657903
Test score is: 0.9864378293166163


C:\Users\WIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.9792226975685764
Accuracy: 0.9864378293166163
Recall: 0.9864378293166163
F1 Score: 0.9815385800393719
{'True Positives (TP)': 162288, 'True Negatives (TN)': 720, 'False Positives (FP)': 2, 'False Negatives (FN)': 17}


Random Forest Classifier

In [34]:
from sklearn.ensemble import RandomForestClassifier

#initialize model
clfr = RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)

#Train model
start_time_train = time.time()
clfr.fit(X_train, y_train)
end_time_train = time.time()
print(f"Training time: {end_time_train - start_time_train:.4f} seconds")

#Predict using the training data
start_time_pred_train = time.time()
y_train_pred = clfr.predict(X_train)
end_time_pred_train = time.time()
print(f"Prediction time on training data: {end_time_pred_train - start_time_pred_train:.4f} seconds")

#Predict using the test data
start_time_pred_test = time.time()
y_test_pred = clfr.predict(X_test)
end_time_pred_test = time.time()
print(f"Prediction time on test data: {end_time_pred_test - start_time_pred_test:.4f} seconds")

# Train and Test scores
print("Train score is:", clfr.score(X_train, y_train))
print("Test score is:", clfr.score(X_test, y_test))

# Calculate metrics
precision_rand = precision_score(y_test, y_test_pred, average='weighted')
accuracy_rand = accuracy_score(y_test, y_test_pred)
recall_rand = recall_score(y_test, y_test_pred, average='weighted')
f1_rand = f1_score(y_test, y_test_pred, average='weighted')
cm_rand = confusion_matrix(y_test, y_test_pred)
r = calculate_tp_tn_fp_fn(cm_rand)

# Display results
print(f"Precision: {precision_rand:.4f}")
print(f"Accuracy: {accuracy_rand:.4f}")
print(f"Recall: {recall_rand:.4f}")
print(f"F1 Score: {f1_rand:.4f}")
#print(f"Confusion Matrix:\n{cm_rand}")
print(r)

Training time: 38.5445 seconds
Prediction time on training data: 9.8666 seconds
Prediction time on test data: 4.7423 seconds
Train score is: 0.9994803531181834
Test score is: 0.9992455237476001


C:\Users\WIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision: 0.9992
Accuracy: 0.9992
Recall: 0.9992
F1 Score: 0.9992
{'True Positives (TP)': 162305, 'True Negatives (TN)': 721, 'False Positives (FP)': 1, 'False Negatives (FN)': 0}


Gradient Boosting

In [35]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Initialize XGBoost Classifier (no need for use_label_encoder, it's deprecated)
clfg = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, eval_metric='mlogloss', random_state=42)

# Encode target labels (y_train) from string to numeric
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [36]:
import time

# Record the start time for training
start_time = time.time()
# Train the model
clfg.fit(X_train, y_train_encoded)
end_time = time.time()
print(f"Training time: {end_time - start_time:.4f} seconds")

Training time: 83.6171 seconds


In [37]:
# Time the prediction process
start_time = time.time()
y_test_pred_xgb_encoded = clfg.predict(X_test)
end_time = time.time()
print(f"Testing time: {end_time - start_time:.4f} seconds")

Testing time: 3.8696 seconds


In [38]:
y_test_pred_xgb = label_encoder.inverse_transform(y_test_pred_xgb_encoded)

In [39]:
#Calculate perfoemance metrics
precision_xgb = precision_score(y_test, y_test_pred_xgb, average='weighted')
accuracy_xgb = accuracy_score(y_test, y_test_pred_xgb)
recall_xgb = recall_score(y_test, y_test_pred_xgb, average='weighted')
f1_xgb = f1_score(y_test, y_test_pred_xgb, average='weighted')
cm_xgb = confusion_matrix(y_test, y_test_pred_xgb)

# Assuming calculate_tp_tn_fp_fn function is defined to take confusion matrix as input
r = calculate_tp_tn_fp_fn(cm_xgb)

# Display results
print(f"Precision (XGBoost): {precision_xgb:.4f}")
print(f"Accuracy (XGBoost): {accuracy_xgb:.4f}")
print(f"Recall (XGBoost): {recall_xgb:.4f}")
print(f"F1 Score (XGBoost): {f1_xgb:.4f}")
#print(f"Confusion Matrix (XGBoost):\n{cm_xgb}")
print(r)

C:\Users\WIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Precision (XGBoost): 0.9997
Accuracy (XGBoost): 0.9997
Recall (XGBoost): 0.9997
F1 Score (XGBoost): 0.9997
{'True Positives (TP)': 162305, 'True Negatives (TN)': 722, 'False Positives (FP)': 0, 'False Negatives (FN)': 0}


Displaying metrics of models

In [41]:
from tabulate import tabulate

# Data for all the models with precomputed values
data = [
    ["Random Forest",accuracy_rand,precision_rand,recall_rand,f1_rand],
    ["Decision Tree",accuracy_dec,precision_rand,recall_dec,f1_dec],
    ["Naive Bayes",accuracy_gaus,precision_gaus,recall_gaus,f1_gaus],
    ["Gradient Boosting",precision_xgb,accuracy_xgb,recall_xgb,f1_xgb]
]

# Define column names
col_names = ["Model", "Accuracy", "Precision", "Recall", "F1 Score"]

# Print the table
print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))

╒═══════════════════╤════════════╤═════════════╤══════════╤════════════╕
│ Model             │   Accuracy │   Precision │   Recall │   F1 Score │
╞═══════════════════╪════════════╪═════════════╪══════════╪════════════╡
│ Random Forest     │   0.999246 │    0.999177 │ 0.999246 │   0.999188 │
├───────────────────┼────────────┼─────────────┼──────────┼────────────┤
│ Decision Tree     │   0.986438 │    0.999177 │ 0.986438 │   0.981539 │
├───────────────────┼────────────┼─────────────┼──────────┼────────────┤
│ Naive Bayes       │   0.856766 │    0.991412 │ 0.856766 │   0.884911 │
├───────────────────┼────────────┼─────────────┼──────────┼────────────┤
│ Gradient Boosting │   0.999733 │    0.999736 │ 0.999736 │   0.999729 │
╘═══════════════════╧════════════╧═════════════╧══════════╧════════════╛


In [43]:
print(type(clfr),clfr)
print(type(clfg),clfg)
print(type(clfd),clfd)
print(type(clfb),clfb)

<class 'sklearn.ensemble._forest.RandomForestClassifier'> RandomForestClassifier(max_depth=10, random_state=42)
<class 'xgboost.sklearn.XGBClassifier'> XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)
<class 'sklearn.tree._classes.DecisionTreeClassifier'> DecisionTreeClassifier(cr

Pickling Models

In [48]:
import joblib

try:
    #save trained models
    joblib.dump(clfr, 'random_forest.pkl')
    joblib.dump(clfg, 'gradient_boosting.pkl')
    joblib.dump(clfd, 'decision_tree.pkl')
    joblib.dump(clfb, 'naive_bayes.pkl')
    print("Models saved successfully!")
except Exception as e:
    print(f'Error in saving models {e}')

Models saved successfully!
